In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from IPython.display import display as Markdown
from tqdm.autonotebook import tqdm as notebook_tqdm

C:\Users\User\AppData\Local\Temp\ipykernel_11328\3632673901.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm as notebook_tqdm


In [21]:
# local_path = "org_gazette.pdf"
local_path = "org_gazette-2-27.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [56]:
# different approach to extract text from PDF

import fitz  # PyMuPDF

def extract_text_by_page(pdf_path):
    # Open the PDF
    doc = fitz.open(pdf_path)
    
    # Extract text from each page
    all_page_text = []
    
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)  # Load page by index
        text = page.get_text("text")  # Extract text using the "text" method
        all_page_text.append(text)  # Add the extracted text to the list
    
    return all_page_text

# Example usage
pdf_path = "org_gazette-2-27.pdf"

# Extract text by page
data = extract_text_by_page(pdf_path)

# Example of how the extracted text might look (you can process this for LLM)
for page_num, page_text in enumerate(data):
    print(f"Page {page_num + 1} Text:")
    print(page_text)  # You can send this text directly to the LLM
    print("=" * 50)

Page 1 Text:
I fldgi ( ^I& fPoh - YS% ,xld m%cd;dka;s%l iudcjd§ ckrcfha w;s úfYI .eiÜ m;%h - 2024'09'27
Part I : Sec. (I) - GAZETTE EXTRAORDINARY OF THE DEMOCRATIC SOCIALIST REPUBLIC OF SRI LANKA - 27.09.2024
2A
Schedule  
1. Minister of Defence
	
Column I	
Column II	
Column III
	
Subjects and Functions	
Departments, Statutory 	
Laws, Acts and Ordinances to be
	
	
Institutions and Public Corporations	
 Implemented
	
1.	 Formulation, implementation, 
monitoring and evaluation of 
policies, strategies, programmes, 
and projects, in relation to 
the subject of Defence, and 
those subjects that come under 
the purview of Departments, 
Statutory Institutions and Public 
Corporations listed in Column 
II based on the national policies 
implemented by the government
	
2.	 Provision of relevant public 
services under the purview of 
the Ministry in an efficient and 
people friendly manner
	
3.	 Reforming all systems and 
procedures using modern 
management techniques and 
technology, thus ensu

In [53]:
# for i, page in enumerate(data):
#     print(f"Page {i+1}:\n{page.page_content[:500]}...")  # Preview first 500 characters of each page
#     print("\n" + "="*40 + "\n")

Page 1:
2A

sCheDuLe

1. minister of Defence

Column I

Column II

Column III

Subjects and Functions

Departments, Statutory Institutions and Public Corporations

Laws, Acts and Ordinances to be Implemented

1. formulation, implementation, monitoring and evaluation of policies, strategies, programmes, and projects, in relation to the subject of Defence, and those subjects that come under the purview of Departments, statutory Institutions and Public Corporations listed in Column II based on the national...




In [22]:
# Preview first page
Markdown(data[0].page_content)

"2A\n\nsCheDuLe\n\n1. minister of Defence\n\nColumn I\n\nColumn II\n\nColumn III\n\nSubjects and Functions\n\nDepartments, Statutory Institutions and Public Corporations\n\nLaws, Acts and Ordinances to be Implemented\n\n1. formulation, implementation, monitoring and evaluation of policies, strategies, programmes, and projects, in relation to the subject of Defence, and those subjects that come under the purview of Departments, statutory Institutions and Public Corporations listed in Column II based on the national policies implemented by the government\n\n2. Provision of relevant public\n\nservices under the purview of the Ministry in an efficient and people friendly manner 3. reforming all systems and\n\nprocedures using modern management techniques and technology, thus ensuring that the functions of the Ministry are fulfilled while eliminating corruption and waste. 4. ensuring national security 5. Maintenance of internal security 6. upgrading the prevailing\n\nintegrated intelligence

In [23]:
# 3. Set the environment variable
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [24]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [26]:
from langchain.schema import Document

# Split and chunk 

#text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
#chunks = text_splitter.split_documents(data)
# chunks = [page.page_content for page in data]
# Wrap the chunks into Document objects
# documents = [Document(page_content=chunk) for chunk in chunks]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=2000)

# Split the document into chunks
chunks = text_splitter.split_documents(data)

In [59]:
# different approach for chunking

def create_overlapping_chunks(data, chunk_size=1, overlap_size=1):
    chunks = []
    
    # Iterate through the data with overlap
    for i in range(0, len(data), chunk_size - overlap_size):
        chunk = ""
        # Add the pages for the current chunk
        for j in range(i, min(i + chunk_size, len(data))):
            chunk += data[j] + "\n"  # Add page text to chunk
        chunks.append(chunk.strip())  # Append the chunk to the list
    
    return chunks

# Create chunks with overlap
chunk_size = 2  # Define chunk size (number of pages per chunk)
overlap_size = 1  # Define overlap size (number of overlapping pages)

chunks = create_overlapping_chunks(data, chunk_size, overlap_size)

# Example output
for idx, chunk in enumerate(chunks):
    print(f"Chunk {idx + 1}:\n{chunk}\n{'='*50}")

documents = [Document(page_content=chunk) for chunk in chunks]

Chunk 1:
I fldgi ( ^I& fPoh - YS% ,xld m%cd;dka;s%l iudcjd§ ckrcfha w;s úfYI .eiÜ m;%h - 2024'09'27
Part I : Sec. (I) - GAZETTE EXTRAORDINARY OF THE DEMOCRATIC SOCIALIST REPUBLIC OF SRI LANKA - 27.09.2024
2A
Schedule  
1. Minister of Defence
	
Column I	
Column II	
Column III
	
Subjects and Functions	
Departments, Statutory 	
Laws, Acts and Ordinances to be
	
	
Institutions and Public Corporations	
 Implemented
	
1.	 Formulation, implementation, 
monitoring and evaluation of 
policies, strategies, programmes, 
and projects, in relation to 
the subject of Defence, and 
those subjects that come under 
the purview of Departments, 
Statutory Institutions and Public 
Corporations listed in Column 
II based on the national policies 
implemented by the government
	
2.	 Provision of relevant public 
services under the purview of 
the Ministry in an efficient and 
people friendly manner
	
3.	 Reforming all systems and 
procedures using modern 
management techniques and 
technology, thus ensuring

In [35]:
print(chunks)

[Document(metadata={'source': 'org_gazette-2-27.pdf'}, page_content='2A\n\nsCheDuLe\n\n1. minister of Defence\n\nColumn I\n\nColumn II\n\nColumn III\n\nSubjects and Functions\n\nDepartments, Statutory Institutions and Public Corporations\n\nLaws, Acts and Ordinances to be Implemented\n\n1. formulation, implementation, monitoring and evaluation of policies, strategies, programmes, and projects, in relation to the subject of Defence, and those subjects that come under the purview of Departments, statutory Institutions and Public Corporations listed in Column II based on the national policies implemented by the government\n\n2. Provision of relevant public\n\nservices under the purview of the Ministry in an efficient and people friendly manner 3. reforming all systems and\n\nprocedures using modern management techniques and technology, thus ensuring that the functions of the Ministry are fulfilled while eliminating corruption and waste. 4. ensuring national security 5. Maintenance of inte

In [60]:
# 5. Try creating the vector database
# vector_db = Chroma.from_documents(
#     documents=chunks,
#     embedding=OllamaEmbeddings(model="nomic-embed-text"),
#     collection_name="local-rag"
# )

vector_db = Chroma.from_documents(
    documents=documents,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)

In [29]:
## Retrieval

from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [30]:
# LLM from Ollama
local_model = "llama3.2"
llm = ChatOllama(model=local_model)

In [80]:
QUERY_PROMPT = ChatPromptTemplate.from_template("""
The document follows a specific structure:
- At the top of each page, there is the name of a ministry in the form "1. Minister of ...." (e.g., "1. Minister of Defence", "2. minister of finance, Economic Development, Policy formulation, Planning and Tourism").
- Note that all ministries have been mentioned in the form "Minister of ..." at the start of each page, if you are required to retrieve all ministries retrieve EVERY ministry mentioned at the top of each page, do not mention duplicates.
- Below the ministry name, a table is provided where the second column lists ALL the departments under that ministry. If departments for that ministry are requested you must list ALL the text within this second column, which can run over several chunks or pages.
- The document has been converted to text using PyMuPDF fitz tool so the tables have been flattened out.
- Note that tables for the same ministry can run over multiple pages and chunks so ensure you check all relevant chunks.

Answer the question based ONLY on the following context from the document:
{context}
Question: {question}
""")

In [67]:
# retriever = MultiQueryRetriever.from_llm(
#     vector_db.as_retriever(), 
#     llm,
#     prompt=QUERY_PROMPT
# )

# Set up the retriever
retriever = vector_db.as_retriever()
#retriever = vector_db.as_retriever(search_kwargs={"k": 20})

# RAG prompt
# template = """Answer the question based ONLY on the following context:
# {context}
# Question: {question}
# """

# prompt = ChatPromptTemplate.from_template(template)

In [71]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | QUERY_PROMPT
    | llm
    | StrOutputParser()
)

In [83]:
chain.invoke("What are the departments under 'minister of finance, Economic Development, Policy formulation, Planning and Tourism' in the document?")

'The text does not explicitly list out all the departments under the "Minister of Finance, Economic Development, Policy Formulation, Planning and Tourism" but it mentions a few tasks assigned to this minister:\n\n1. Financial administration of Public Corporations, Statutory Boards and Government Owned Companies\n2. Development of Colombo Port City Special Economic Zone with the objective of national welfare and economic development\n3. Promotion of regional trade and investment cooperation through policy formulation, strategies, programmes and projects\n\nThe rest of the tasks are assigned to other ministers or departments.\n\nHere is a list of some departments/states that have been mentioned in the text under this minister:\n\n1. Colombo Port City Economic Commission\n2. Public Debt Management Department\n3. Greater Colombo Economic Commission (though it seems like there might be some overlap between this and \'Development of Colombo Port City Special Economic Zone\')\n\nPlease note, 

In [46]:
# Delete all collections in the db
#vector_db.delete_collection()